In [2]:
import cv2
import os
import numpy as np
import time


In [47]:
import cv2
import os
import imutils

# Cambia 'emotionName' según la emoción que quieras capturar
emotionName = 'Felicidad'
# emotionName = 'Sorpresa'
# emotionName = 'Tristeza'

dataPath = './dataset'  # Cambia a la ruta donde hayas almacenado Data
emotionsPath = dataPath + '/' + emotionName

if not os.path.exists(emotionsPath):
    print('Carpeta creada: ', emotionsPath)
    os.makedirs(emotionsPath)

# Obtener el número de archivos existentes en la carpeta para continuar desde ahí
count = len(os.listdir(emotionsPath))

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if ret == False: break
    frame = imutils.resize(frame, width=640)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = frame.copy()
    faces = faceClassif.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        rostro = auxFrame[y:y + h, x:x + w]
        rostro = cv2.resize(rostro, (150, 150), interpolation=cv2.INTER_CUBIC)
        cv2.imwrite(emotionsPath + '/rostro_{}.jpg'.format(count), rostro)
        count += 1
    
    cv2.imshow('frame', frame)
    k = cv2.waitKey(1)
    if k == 27 or count >= 10:  # Presiona 'ESC' para salir o captura 200 imágenes
        break

cap.release()
cv2.destroyAllWindows()


In [52]:

def obtenerModelo(method, facesData, labels):
    if method == 'EigenFaces': 
        emotion_recognizer = cv2.face.EigenFaceRecognizer_create()
    if method == 'FisherFaces': 
        emotion_recognizer = cv2.face.FisherFaceRecognizer_create()
    if method == 'LBPH': 
        emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Entrenando el reconocedor de rostros
    print("Entrenando ( " + method + " )...")
    inicio = time.time()
    emotion_recognizer.train(facesData, np.array(labels))
    tiempoEntrenamiento = time.time() - inicio
    print("Tiempo de entrenamiento ( " + method + " ): ", tiempoEntrenamiento)
    
    # Almacenando el modelo obtenido
    emotion_recognizer.write("modelo" + method + ".xml")

dataPath = './dataset'  
emotionsList = ['Tristeza', 'Felicidad', 'Sorpresa']
print('Lista de emociones: ', emotionsList)

labels = []
facesData = []
label = 0

for nameDir in emotionsList:
    emotionsPath = dataPath + '/' + nameDir
    for fileName in os.listdir(emotionsPath):
        labels.append(label)
        facesData.append(cv2.imread(emotionsPath + '/' + fileName, 0))
    label += 1

# obtenerModelo('EigenFaces', facesData, labels)
# obtenerModelo('FisherFaces', facesData, labels)
obtenerModelo('LBPH', facesData, labels)


Lista de emociones:  ['Tristeza', 'Felicidad', 'Sorpresa']
Entrenando ( LBPH )...
Tiempo de entrenamiento ( LBPH ):  3.447150230407715


In [57]:
import cv2
import os
import numpy as np


# method = 'EigenFaces'
# method = 'FisherFaces'
method = 'LBPH'
if method == 'EigenFaces': 
    emotion_recognizer = cv2.face.EigenFaceRecognizer_create()
if method == 'FisherFaces': 
    emotion_recognizer = cv2.face.FisherFaceRecognizer_create()
if method == 'LBPH': 
    emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()

emotion_recognizer.read('modelo' + method + '.xml')

dataPath = './dataset'  
emotionsList = ['Tristeza', 'Felicidad', 'Sorpresa']
print('Lista de emociones: ', emotionsList)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = gray.copy()
    nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])
    faces = faceClassif.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        rostro = auxFrame[y:y + h, x:x + w]
        rostro = cv2.resize(rostro, (150, 150), interpolation=cv2.INTER_CUBIC)
        result = emotion_recognizer.predict(rostro)
        print('result:', result)
        cv2.putText(frame, '{}'.format(result), (x, y - 5), 1, 1.3, (255, 255, 0), 1, cv2.LINE_AA)
        
        if method == 'EigenFaces':
            if result[1] < 5700:
                cv2.putText(frame, '{}'.format(emotionsList[result[0]]), (x, y - 25), 2, 1.1, (0, 255, 0), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                nFrame = cv2.hconcat([frame])
            else:
                cv2.putText(frame, 'No identificado', (x, y - 20), 2, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])
        
        if method == 'FisherFaces':
            if result[1] < 500:
                cv2.putText(frame, '{}'.format(emotionsList[result[0]]), (x, y - 25), 2, 1.1, (0, 255, 0), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                nFrame = cv2.hconcat([frame])
            else:
                cv2.putText(frame, 'No identificado', (x, y - 20), 2, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])
        
        if method == 'LBPH':
            if result[1] < 100:
                cv2.putText(frame, '{}'.format(emotionsList[result[0]]), (x, y - 25), 2, 1.1, (0, 255, 0), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                nFrame = cv2.hconcat([frame])
            else:
                cv2.putText(frame, 'No identificado', (x, y - 20), 2, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])
    
    cv2.imshow('nFrame', nFrame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


Lista de emociones:  ['Tristeza', 'Felicidad', 'Sorpresa']
result: (2, 86.08679458623621)
result: (2, 86.27973220407515)
result: (2, 87.32637142339111)
result: (2, 84.04475370770179)
result: (2, 85.13590989324804)
result: (2, 86.29129121041613)
result: (2, 85.70599851257833)
result: (2, 85.93068547778729)
result: (2, 86.23057850188262)
result: (2, 85.54700914133063)
result: (2, 85.16463968655671)
result: (2, 83.06012828301792)
result: (2, 84.71612283308745)
result: (2, 84.68983848441765)
result: (2, 85.38005247812866)
result: (2, 86.61691943475059)
result: (2, 88.86356987122588)
result: (2, 93.45808879996405)
result: (2, 89.73123979034568)
result: (2, 83.10145612068844)
result: (2, 83.403353949949)
result: (2, 83.54368685460149)
result: (2, 82.92511330268005)
result: (2, 84.20461217123848)
result: (2, 85.8243815451984)
result: (2, 82.52569880263702)
result: (2, 82.9405209350493)
result: (2, 80.71561041304223)
result: (2, 83.17006536731638)
result: (2, 84.66552033679234)
result: (2, 87.

Inicializando el detector de rostros con Haar Cascade...
Detector de rostros inicializado.
Iniciando la captura de video...
Cámara abierta exitosamente.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
Rostros detectados: 0
Frame procesado.
Procesando frame...
R